# S&P500 trend follwing on mt5 

Optimizing the indicators to find the best blend



### Result



In [7]:
import MetaTrader5 as mt5
import pandas as pd
import ta
import time
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from datetime import datetime
mt5.initialize()

True

In [9]:
symbol = 'SP500USD'
timeframe = mt5.TIMEFRAME_M15
start_date = datetime(2023,1,1)
end_time = datetime.now()

def get_data(symbol, timeframe, start_date, end_time):
    rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_time)
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    df.columns = df.columns.str.capitalize()
    return df

us500 = get_data(symbol, timeframe, start_date, end_time)
us500.dropna(inplace=True)


In [10]:
class BollingerBandBuy(Strategy):
    initsize = 5.0
    mysize = initsize
    bb_window = 25
    
    def init(self):
        super().init()
        self.lower_bb = self.I(ta.volatility.bollinger_lband, pd.Series(self.data.Close), window=self.bb_window)
        
        
    def next(self):
        super().next()
            
        if (self.data.Close[-1] > self.lower_bb[-1]) and (self.data.Close[-2] <= self.lower_bb[-2]):
            sl1 = self.data.Close[-1] - 250e-1
            tp1 = self.data.Close[-1] + 550e-1
            self.buy(sl=sl1, tp=tp1, size=self.mysize)      
            
# Assuming us500 is defined elsewhere
bt = Backtest(us500, BollingerBandBuy, cash=10000, exclusive_orders=True, margin=1/100)

output = bt.run()
output


Start                     2023-01-03 01:00:00
End                       2024-04-23 14:00:00
Duration                    476 days 13:00:00
Exposure Time [%]                   84.299738
Equity Final [$]                      15293.9
Equity Peak [$]                      15709.55
Return [%]                             52.939
Buy & Hold Return [%]               30.521964
Return (Ann.) [%]                     38.2449
Volatility (Ann.) [%]               29.949228
Sharpe Ratio                         1.276991
Sortino Ratio                        2.726214
Calmar Ratio                         2.310892
Max. Drawdown [%]                  -16.549841
Avg. Drawdown [%]                   -0.626622
Max. Drawdown Duration      119 days 13:45:00
Avg. Drawdown Duration        1 days 14:14:00
# Trades                                  866
Win Rate [%]                          35.2194
Best Trade [%]                       1.428285
Worst Trade [%]                     -0.657816
Avg. Trade [%]                    

In [11]:


# Run optimization
optimization_results = bt.optimize(bb_window=range(5, 100, 5), maximize='Sharpe Ratio')

optimization_results

  0%|          | 0/10 [00:00<?, ?it/s]

Start                     2023-01-03 01:00:00
End                       2024-04-23 14:00:00
Duration                    476 days 13:00:00
Exposure Time [%]                   79.270288
Equity Final [$]                     15223.85
Equity Peak [$]                      15711.95
Return [%]                            52.2385
Buy & Hold Return [%]               30.521964
Return (Ann.) [%]                   37.539158
Volatility (Ann.) [%]               26.856854
Sharpe Ratio                          1.39775
Sortino Ratio                        2.992922
Calmar Ratio                         2.755031
Max. Drawdown [%]                  -13.625676
Avg. Drawdown [%]                   -0.590108
Max. Drawdown Duration      124 days 23:15:00
Avg. Drawdown Duration        1 days 15:03:00
# Trades                                  754
Win Rate [%]                        36.074271
Best Trade [%]                       1.446525
Worst Trade [%]                     -0.657816
Avg. Trade [%]                    